In [1]:
#importing libraries

import random as rd
import hashlib
import phe
import rsa

In [2]:
#declaring of varibles
numberOfExperts = 3
numberOfCandidates = 5
#collection of homomophic public and private keys
dictionaryOfExpertPublicKeys = {}
dictionaryOfExpertPrivateKeys = {}

dictionaryOfCandidatesPublicKeys = {}
dictionaryOfCandidatesPrivateKeys = {}

#Variables for third framework@
dictionaryOfRandomNumberPerExpert = {}
dictionaryOftheSumOfRandomNumbermappedToEveryCandidate = {}
dictionaryOfRandomNumberGatheredFromOtherExperts = {}
dictionaryOfRandomNumberMappedToEveryCandidate = {}

#Collection of rsa public and private keys for digital signature
dictOfrsaPubKey = {}
dictOfrsaPriKey = {}
marksToCandidates = [[0 for i in range(numberOfCandidates)] for j in range(numberOfExperts)]


In [3]:
#this part of the code create the public and private homomorphic key pairs

for i in range(numberOfExperts):
    x,y = phe.generate_paillier_keypair(n_length=128)
    dictionaryOfExpertPublicKeys[i] = x
    dictionaryOfExpertPrivateKeys[i] = y
    

for i in range(numberOfCandidates):
    x,y = phe.generate_paillier_keypair(n_length=128)
    dictionaryOfCandidatesPublicKeys[i]= x
    dictionaryOfCandidatesPrivateKeys[i] = y 

publicKeyEmployer,privateKeyEmployer = phe.generate_paillier_keypair(n_length=128)


In [4]:
#@code to generate rsa public and private keys
for i in range(numberOfExperts):
    (pubkey, privkey) = rsa.newkeys(512)
    dictOfrsaPubKey[i],dictOfrsaPriKey[i] = pubkey, privkey 


In [5]:
# function to create digital signature
def createDigitalSignature(message,i):
    privkey = dictOfrsaPriKey[i]
    temp = str(message).encode()
    signature = rsa.sign(temp, privkey, 'SHA-1')
    return signature

In [6]:
#this part of the code finds the highest marks gaining candidate id this acts as the candidate manager for the first frame work
def candidateManagerFrameWork1BestCandidate():
    array  = []
    for i in range(len(marksToCandidates[0])):
        sum = 0
        for j in range(len(marksToCandidates)):
            sum = sum + marksToCandidates[j][i][0]
        print(sum)
        array.append(sum)
        
    print("hihgest marks gaining candidate =",array.index( max(array)))    


In [7]:
#this part of the code defines the first frame work
def framreWork1():
    global marksToCandidates
    for i in range(numberOfExperts):
        for j in range(numberOfCandidates):
            mark = rd.randint(0,10)
            marksToCandidates[i][j] = [mark,createDigitalSignature(mark,i)]
    candidateManagerFrameWork1BestCandidate()

In [8]:
# framreWork1()
# print(marksToCandidates)

In [9]:
# tjhis part of the code works as the employer of the seconde frame work who finds the est candidate
def employerFindingBestCandidate(array):
    array = [privateKeyEmployer.raw_decrypt(x) for x in array]
    print(array.index(max(array)))
    print(array)

In [10]:
# this part of the code works as the candidate manager who adds the marks by multiplication of the encrytpion and than passes them to the employer
def candidateManagerFrameWork2BestCandidate():
    array  = []
    for i in range(len(marksToCandidates[0])):
        sum = 1
        for j in range(len(marksToCandidates)):
            sum = sum * marksToCandidates[j][i][0]
        array.append(sum)
    employerFindingBestCandidate(array)
    

In [11]:
def frameWork2():
    global marksToCandidates
    #this part of code simulates the expert giving marks to the candidates and storing them in form of matrix
    for i in range(numberOfExperts):
        for j in range(numberOfCandidates):
            mark = rd.randint(0,10)
            encryptedMark = publicKeyEmployer.raw_encrypt(mark)
            marksToCandidates[i][j] = [encryptedMark,createDigitalSignature(mark,i)]
    candidateManagerFrameWork2BestCandidate()
    

In [12]:
# frameWork2()


In [13]:
def generateRandomNumberForEveryExpert():
    # global dictionaryOfRandomNumberPerExpert
    generatedRandomNumbers = [rd.randint(0,100) for x in range(numberOfExperts)]
    # print(generatedRandomNumbers)
    return generatedRandomNumbers
    # print(dictionaryOfSumGeneratedPerRandomNumber)

In [14]:
def transferingTheRandomNumberOfEveryExpert():
        for i in range(numberOfExperts):
                dictionaryOfRandomNumberGatheredFromOtherExperts[i] = [dictionaryOfRandomNumberPerExpert[k][i] for k in range(numberOfExperts) if k != i]

In [15]:
def forwardTheRespectiveRandomNumbers(idR,generatedRandomNumbers):
    encryptedRandomNumber = dictionaryOfExpertPublicKeys[idR].raw_encrypt(generatedRandomNumbers[idR])
    message = encryptedRandomNumber
    return message
# ,createDigitalSignature(encryptedRandomNumber,idS)

In [16]:
# def expertsCalculatingTheHiddenScore(dictionaryOfRandomNumberMappedToEveryCandidate):
#     for i in range(numberOfCandidates):
#         for j in range(numberOfExperts):
#             mark = rd.randint(0,10)
#             hiddenScore = mark + dictionaryOfRandomNumberMappedToEveryCandidate[i][j][j]
#             sum = 0
#             for k in range(numberOfExperts):
#                     encryptedRandomNumber = dictionaryOfRandomNumberMappedToEveryCandidate[i][k][j]
#                     sum  = sum + dictionaryOfExpertPrivateKeys[j].raw_decrypt(encryptedRandomNumber)
#             hiddenScore = hiddenScore - sum
#             message = hiddenScore
#             marksToCandidates[j][i] = message
#             # ,createDigitalSignature(hiddenScore,j)

In [17]:
def performSMPCBetweenExperts():
    #this part of the code simulates the part where expert follow the smpc process
    #the random number assgned by all the experts has been mapped for every candidate
    
    for i in range(numberOfCandidates):
        matrixOfForwardedMessages = [[0 for i in range(numberOfExperts)] for j in range(numberOfExperts)]
        sumList = []
        for j in range(numberOfExperts):
            generatedRandomNumbers = generateRandomNumberForEveryExpert()
            sumOfTheRandomNumbers = sum(generatedRandomNumbers) 
            sumList.append(sumOfTheRandomNumbers)

            for k in range(numberOfExperts):
                if(j != k):
                    message = forwardTheRespectiveRandomNumbers(k,generatedRandomNumbers)
                    matrixOfForwardedMessages[j][k] = message
                else:
                    matrixOfForwardedMessages[j][k] = generatedRandomNumbers[j]
        dictionaryOfRandomNumberMappedToEveryCandidate[i] = matrixOfForwardedMessages
        dictionaryOftheSumOfRandomNumbermappedToEveryCandidate[i] = sumList
    print(dictionaryOfRandomNumberMappedToEveryCandidate)
    print(dictionaryOftheSumOfRandomNumbermappedToEveryCandidate)
    # print(dictionaryOfSumGeneratedPerRandomNumber)
    

    

In [18]:
def generateHiddenScoreFrameWork3():
    global marksToCandidates
    for i in range(numberOfCandidates):
        for j in range(numberOfExperts):
            mark = rd.randint(0,100)
            hiddenScore = mark + dictionaryOftheSumOfRandomNumbermappedToEveryCandidate[i][j]
            sum  = 0
            for k in range(numberOfExperts):
                if(j != k):
                    encryptedRandomNumber = dictionaryOfRandomNumberMappedToEveryCandidate[i][k][j]
                    # print(encryptedRandomNumber)
                    sum  = sum + dictionaryOfExpertPrivateKeys[j].raw_decrypt(encryptedRandomNumber)
                else: 
                    sum = sum + dictionaryOfRandomNumberMappedToEveryCandidate[i][j][j]
            hiddenScore  = hiddenScore -sum
            print(hiddenScore)
            marksToCandidates[j][i] = hiddenScore
            


In [19]:
def candidateManagerFrameWork3BestCandidate():
    array = []
    for i in range(numberOfCandidates):
        sum = 0
        for j in range(numberOfExperts):
            cummulativeScore = marksToCandidates[j][i]
            sum  = sum + cummulativeScore
        array.append(sum)
    print(array)
    print(array.index(max(array)))

In [20]:
def frameWork3():
    performSMPCBetweenExperts()
    generateHiddenScoreFrameWork3()
    candidateManagerFrameWork3BestCandidate()
    
        
    #     dictionaryOfRandomNumberMappedToEveryCandidate[i]   = matrixOfForwardedMessages  
    #     dictionaryOfSumGeneratedPerRandomNumber[i] = matrixOfTheSumOfRandomnumbers  
    # expertsCalculatingTheHiddenScore(dictionaryOfRandomNumberMappedToEveryCandidate)     
    # # candidateManagerFrameWork3BestCandidate()
    

In [21]:
frameWork3()

{0: [[84, 39650820178489163978911462227970469632285525931661010909482542901810259212969, 2812203087293696605696880361758387165322125311664307991417783619868272875383], [28541455336169821228538024779382266622961428178990207995024362907913777745918, 40, 51604464567749000975040297738299234862853249847487755615103299951182892953626], [16878508182111693746803470223207732056747302138558537468714586571819927711782, 36676613719992085529339810157007624914030818490077640744693981384114498969516, 30]], 1: [[84, 1523363848499017928764023187219362157424803964253472717580373803542048523571, 41831495367525658238444225783951693102861260159066760640442377350724586617420], [3029090858238835826933398083640186866467598866705668285271368511779998472757, 4, 38167406906376186798365982341887262360397143035946037584142683883395075721910], [25866871368653147673156089818591289194428400574261882114824347079075796504886, 46510354052592453093286469931889982525953650119273735117679259346816691390001, 40]], 2: [[8, 2

In [22]:
def generateHiddenScoreFrameWork4():
    global marksToCandidates
    for i in range(numberOfCandidates):
        for j in range(numberOfExperts):
            mark = rd.randint(0,100)
            hiddenScore = mark + dictionaryOftheSumOfRandomNumbermappedToEveryCandidate[i][j]
            sum  = 0
            for k in range(numberOfExperts):
                if(j != k):
                    encryptedRandomNumber = dictionaryOfRandomNumberMappedToEveryCandidate[i][k][j]
                    # print(encryptedRandomNumber)
                    sum  = sum + dictionaryOfExpertPrivateKeys[j].raw_decrypt(encryptedRandomNumber)
                else: 
                    sum = sum + dictionaryOfRandomNumberMappedToEveryCandidate[i][j][j]
            hiddenScore  = hiddenScore -sum
            # print(hiddenScore)
            marksToCandidates[j][i] = publicKeyEmployer.raw_encrypt(hiddenScore)

In [23]:
def employerFindsTheBestCandidate(array):
    marksOfCandidates = [privateKeyEmployer.raw_decrypt(x) for x in array]
    print(marksOfCandidates)


In [24]:
def candidateManagerFrameWork4BestCandidate():
    array = []
    for i in range(numberOfCandidates):
        sum = 1
        for j in range(numberOfExperts):
            cummulativeScore = marksToCandidates[j][i]
            sum  = sum * cummulativeScore
        array.append(sum)
    employerFindsTheBestCandidate(array)


In [25]:
def framework4():
    performSMPCBetweenExperts()
    generateHiddenScoreFrameWork4()
    candidateManagerFrameWork4BestCandidate()


In [26]:
framework4()

{0: [[100, 7276460624286034924825259370524207079550693854564177156372032118919079796570, 12095192195885211508699910700320986728985557459681919709325881221485109415634], [20447552542890760478132435010417538580904160689123070586921624442561182549074, 38, 8595648749093165086791591161289125675633153862091767799566790741707702206451], [20574902939092473232945195792532353946393257406329217396955956056002046671168, 58865829948380994029944540312766186907553847985360945569653669544951044889126, 9]], 1: [[24, 38344380462728175369503084136659251290646279011079772482426315949457956616682, 2343232222043661938603249960647595700200600172763806091732729564950132761073], [24324040777111709356509029810141112755710846969329926093334815877997662118112, 36, 23181548150367472357693754297191226037318715948818699015656857574393086469977], [29555085910970936319335971581791637507686452857810418612366162925362573219657, 75559487064550231128168081029618080851134350740044844234372662122627610701007, 68]], 2: [[9, 